# Props Data Organizer

## Definitions of Formulas Used in This Notebook

### Estimated Value (EV)
EV is the average amount you can expect to win or lose per bet if you placed the same bet many times. It helps identify profitable betting opportunities by comparing the expected return to the risk involved.

**Formula:**
$$
\text{EV} = (\text{Probability of Winning} \times \text{Profit if Win}) - (\text{Probability of Losing} \times \text{Loss if Lose})
$$

### Kelly Criterion
The Kelly Criterion is a formula used to determine the optimal size of a series of bets. It aims to maximize the logarithm of wealth, balancing the trade-off between risk and reward. The formula considers both the probability of winning and the odds offered, guiding you on how much of your bankroll to wager on each bet.

**Formula:**
$$
\text{Kelly Fraction} = \frac{(\text{Probability of Winning} \times (\text{Odds} + 1)) - 1}{\text{Odds}}
$$

In [1]:
import pandas as pd 
import numpy as np
import time
import requests
from NBAData.gambling import *
from datetime import datetime
from Models.xgboost_prediction import *

today = datetime.now()
formatted_date = today.strftime("%m_%d_%y")

### Historical data from the odds api, but i have to pay. Worst case ill pay, if im not able to scrap from betting pros

In [2]:
base_url = "https://api.the-odds-api.com/v4/sports/basketball_nba/events/"
API_KEY = '8aa8bb1fc5d56c98b45d6f3f58beab70'
Sport = 'basketball_nba'
Regions = 'us'
Market = 'h2h,odds'
Odds_format = 'american'
Date_format = 'iso' 
date = '2021-10-22T22:45:00Z'
id = '34e829d65b6a0cbb49573338a86772ec'

url = (
    f"https://api.the-odds-api.com/v4/historical/sports/{Sport}/events"
    f"?apiKey={API_KEY}"
    f"&date={date}"
    f"&regions={Regions}"
)

response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to get events: {response.status_code}, {response.text}")
else:
    events = response.json()
    for event in events:
        print(event['id'], event['commence_time'], event['home_team'], event['away_team'])

Failed to get events: 401, {"message":"Historical odds are only available on paid usage plans. See usage plans at https://the-odds-api.com","error_code":"HISTORICAL_UNAVAILABLE_ON_FREE_USAGE_PLAN","details_url":"https://the-odds-api.com/liveapi/guides/v4/api-error-codes.html#historical-unavailable-on-free-usage-plan"}



### Grabs players odds for the day (US all boookmakers, DFS is prizepicks and underdogs)

In [1]:
from NBAPropFinder.NBAPropFinder import NBAPropFinder

nba_props = NBAPropFinder(region='us_dfs')
prizePicks = nba_props.dataframe
prizePicks.head(10)

Scraping Odds API...


WebDriverException: Message: Process unexpectedly closed with status 0


### Single Bets from bookmakers that dont include prizePicks or UnderDogs

In [3]:
model = load_xgboost_model('PTS')
prizePicksProps = loadPrizePicksProps(date_str='06_17_25', prop_type='player_points')
bookmakers = pd.read_csv(f'PROPS_DATA/Playoffs_US(06_22_25).csv')
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_PTS_FEATURES.csv')
games = get_espn_games(date_str='20250619')
opponent = findOPP('Shai Gilgeous-Alexander', data, games)
pred = make_prediction('Shai Gilgeous-Alexander', opponent, model, data, prizePicksProps, games, 0, 0, 0, 'PTS')
results = monte_carlo_prop_simulation(pred['player'], data, pred['predicted_stat'], pred['prop_line'],'PTS')
results



{'mean_prediction': 30,
 'std_used': 9.0,
 'prob_over': 0.3489,
 'prob_under': 0.6511,
 'confidence_interval': (12.57731457283782, 47.47542720803084)}

In [ ]:
# Dictionary mapping prop categories to their stat columns
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    # 'player_threes': 'FG3M',
    # 'player_blocks': 'BLK',
    # 'player_steals': 'STL',
    # 'player_field_goals': 'FGM',
    # 'player_threes': 'FG3M',
    # 'player_frees_made': 'FTM',
    # 'player_frees_attempts': 'FTA',
    # 'player_turnovers': 'TOV',
    # 'player_points_rebounds_assists': 'PTS+REB+AST',
    # 'player_points_rebounds': 'PTS+REB',
    # 'player_points_assists': 'PTS+AST',
    # 'player_rebounds_assists': 'REB+AST',
    # 'player_blocks_steals': 'BLK+STL'
}
models = {
    'PTS': load_xgboost_model('PTS'),
    'REB': load_xgboost_model('REB'),
    'AST': load_xgboost_model('AST'),
}
all_results = []

for category, stat in propDict.items():
    print(f"Processing {category}...")
    data = pd.read_csv(f'PLAYOFF_DATA/PLAYOFFS_25_{stat}_FEATURES.csv')
    results = single_bet(data, bookmakers, models, games, category=category, stat_line=stat)
    all_results.append(results)

combined_results = pd.concat(all_results, ignore_index=True)

final_results = combined_results.sort_values(by='EV', ascending=False).reset_index(drop=True)

print("\nTop 15 highest EV bets across all prop types:")
final_results.head(15)

Processing player_points...
Processing player_rebounds...
Processing player_assists...

Top 15 highest EV bets across all prop types:


,NAME,BOOKMAKER,CATEGORY,LINE,OVER/UNDER,ODDS,FAIR ODDS,SIM PROB,EV,KELLY CRITERION
0,Aaron Nesmith,Bovada,player_points,7.5,Under,215,251,0.925,191.47,0.9250
1,Andrew Nembhard,Bovada,player_points,7.5,Under,215,251,0.844,165.95,0.8436
2,Tyrese Haliburton,Bovada,player_assists,5.5,Under,205,239,0.828,152.54,0.8272
3,Myles Turner,Bovada,player_points,9.5,Under,215,251,0.779,145.26,0.7776
4,Ben Sheppard,DraftKings,player_assists,0.5,Over,165,190,0.837,121.86,0.8362
5,Aaron Nesmith,Bovada,player_rebounds,3.5,Under,215,251,0.693,118.39,0.6919
6,Aaron Nesmith,Bovada,player_points,8.5,Under,150,174,0.871,117.72,0.8700
7,Tyrese Haliburton,Bovada,player_points,12.5,Under,190,228,0.674,95.52,0.6725
8,Andrew Nembhard,Bovada,player_points,8.5,Under,150,174,0.780,94.98,0.7784
9,Pascal Siakam,Bovada,player_points,16.5,Under,170,199,0.688,85.71,0.6860


In [ ]:
model = load_xgboost_model('PTS')
bookmakers = pd.read_csv(f'PROPS_DATA/Playoffs_DFS(06_21_25).csv')
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_PTS_FEATURES.csv')
games = get_espn_games(date_str='20250619')
# opponent = findOPP('Shai Gilgeous-Alexander', data, games)
# pred = make_prediction('Shai Gilgeous-Alexander', opponent, model, data, prizePicksProps, games, 0, 0, 0, 'PTS')
# results = monte_carlo_prop_simulation(pred['player'], data, pred['predicted_stat'], pred['prop_line'],'PTS')

propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    # 'player_threes': 'FG3M',
    # 'player_blocks': 'BLK',
    # 'player_steals': 'STL',
    # 'player_field_goals': 'FGM',
    # 'player_threes': 'FG3M',
    # 'player_frees_made': 'FTM',
    # 'player_frees_attempts': 'FTA',
    # 'player_points_rebounds_assists': 'PTS+REB+AST',
    # 'player_points_rebounds': 'PTS+REB',
    # 'player_points_assists': 'PTS+AST',
    # 'player_rebounds_assists': 'REB+AST',
    # 'player_turnovers': 'TOV',
}
models = {
    'PTS': load_xgboost_model('PTS'),
    'REB': load_xgboost_model('REB'),
    'AST': load_xgboost_model('AST'),
}
duo_results = prizePicksPairsEV(
    prizePicks=bookmakers,
    propDict=propDict,
    models=models,
    games=games,
    simulations=10000,
    stake=100,
    payout=300
)
trio_results = prizePicksTriosEV(
    prizePicks=bookmakers,
    propDict=propDict,
    models=models,
    games=games,
    simulations=10000,
    stake=100,
    payout=600
)

Loading datasets and generating valid combinations...
Loaded dataset for REB
Loaded dataset for PTS
Loaded dataset for AST
Processing 6334 combinations with 8 threads...
Completed 100/6334 combinations
Completed 200/6334 combinations
Completed 300/6334 combinations
Completed 400/6334 combinations
Completed 500/6334 combinations
Completed 600/6334 combinations
Completed 700/6334 combinations
Completed 800/6334 combinations
Completed 900/6334 combinations
Completed 1000/6334 combinations
Completed 1100/6334 combinations
Completed 1200/6334 combinations
Completed 1300/6334 combinations
Completed 1400/6334 combinations
Completed 1500/6334 combinations
Completed 1600/6334 combinations
Completed 1700/6334 combinations
Completed 1800/6334 combinations
Completed 1900/6334 combinations
Completed 2000/6334 combinations
Completed 2100/6334 combinations
Completed 2200/6334 combinations
Completed 2300/6334 combinations
Completed 2400/6334 combinations
Completed 2500/6334 combinations
Completed 2600

In [24]:
trio_results.sort_values(by='KELLY CRITERION', ascending=False).head(15)

,PLAYER 1,CATEGORY 1,STAT TYPE 1,PLAYER 1 LINE,PLAYER 1 PREDICTION,PLAYER 2,CATEGORY 2,STAT TYPE 2,PLAYER 2 LINE,PLAYER 2 PREDICTION,PLAYER 3,CATEGORY 3,STAT TYPE 3,PLAYER 3 LINE,PLAYER 3 PREDICTION,TYPE,EV,PROBABILITY,KELLY CRITERION
2879,Obi Toppin,player_rebounds,REB,5.0,3,Alex Caruso,player_rebounds,REB,3.5,2,Pascal Siakam,player_rebounds,REB,7.5,6,UNDER/UNDER/UNDER,368.35,0.7806,0.7801
2757,Obi Toppin,player_rebounds,REB,5.0,3,Kenrich Williams,player_assists,AST,0.5,1,Pascal Siakam,player_rebounds,REB,7.5,6,UNDER/OVER/UNDER,325.85,0.7097,0.7092
2956,Obi Toppin,player_rebounds,REB,5.0,3,Chet Holmgren,player_rebounds,REB,10.0,7,Pascal Siakam,player_rebounds,REB,7.5,6,UNDER/UNDER/UNDER,317.28,0.6955,0.6949
3802,Kenrich Williams,player_assists,AST,0.5,1,Alex Caruso,player_rebounds,REB,3.5,2,Pascal Siakam,player_rebounds,REB,7.5,6,OVER/UNDER/UNDER,302.89,0.6715,0.6708
2747,Obi Toppin,player_rebounds,REB,5.0,3,Kenrich Williams,player_assists,AST,0.5,1,Alex Caruso,player_rebounds,REB,3.5,2,UNDER/OVER/UNDER,301.68,0.6695,0.6688
3020,Obi Toppin,player_rebounds,REB,5.0,3,Pascal Siakam,player_rebounds,REB,7.5,6,Aaron Wiggins,player_rebounds,REB,2.5,1,UNDER/UNDER/UNDER,298.11,0.6635,0.6628
5168,Alex Caruso,player_rebounds,REB,3.5,2,Chet Holmgren,player_rebounds,REB,10.0,7,Pascal Siakam,player_rebounds,REB,7.5,6,UNDER/UNDER/UNDER,294.65,0.6577,0.6571
2873,Obi Toppin,player_rebounds,REB,5.0,3,Alex Caruso,player_rebounds,REB,3.5,2,Chet Holmgren,player_rebounds,REB,10.0,7,UNDER/UNDER/UNDER,292.05,0.6534,0.6527
2980,Obi Toppin,player_rebounds,REB,5.0,3,Myles Turner,player_rebounds,REB,5.0,4,Pascal Siakam,player_rebounds,REB,7.5,6,UNDER/UNDER/UNDER,273.06,0.6218,0.6210
5238,Alex Caruso,player_rebounds,REB,3.5,2,Pascal Siakam,player_rebounds,REB,7.5,6,Aaron Wiggins,player_rebounds,REB,2.5,1,UNDER/UNDER/UNDER,272.85,0.6214,0.6207
